In [1]:
%matplotlib inline

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
import h5py
import numpy.ma as ma
from scipy.signal import butter, lfilter, freqz
from scipy.interpolate import interp1d

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
#Opening h5 data file, reading data into arrays, then closing file
file_name = '20120123.004_lp_5min-cal.h5'
hf = h5py.File(file_name, 'r')

altitude = np.array(hf['NeFromPower/Altitude'])
Ne_NoTr = np.array(hf['NeFromPower/Ne_NoTr'])
dtime = np.array(hf['Time/dtime'])

hf.close()

#Printing shape of arrays
print 'Altiude Shape: ', altitude.shape
print 'Ne_NoTr Shape: ', Ne_NoTr.shape
print 'dtime Shape: ', dtime.shape

Altiude Shape:  (4, 180)
Ne_NoTr Shape:  (159, 4, 180)
dtime Shape:  (159, 2)


In [4]:
Ne_alt_0_p_0 = Ne_NoTr[:,0,0] # 1-D Ne array for altitude 0 and pole 0
Ne_alt_0_p_0_m1 = ma.masked_less_equal(Ne_alt_0_p_0,0) # masking negative electron densities
Ne_alt_0_p_0_m2 = np.ma.masked_invalid(Ne_alt_0_p_0_m1) # masking NaN values

#Adding 24 to  all values in time array after UT resets to zero
dtime_shift = np.copy(dtime)
dtime_shift[111,1] += 24
dtime_shift[112:,:]+= 24
#Creating mean time array
dtime_shift_mean = np.mean(dtime_shift,axis=1)
#Matrix of indexes for nonzero (i.e. nonmasked) elements in Ne array
Ne_index = ma.nonzero(Ne_alt_0_p_0_m2)

time_mask_removed = np.copy(dtime_shift_mean)
# Setting masked values in time array to 0
for i in range(159):
    if np.isin(i,Ne_index) == False:
      time_mask_removed[i] = 0
    
# Removing zero values from time array   
time_interp = time_mask_removed[np.nonzero(time_mask_removed)] 
# Removing masked values from Ne array
Ne_interp = ma.compressed(Ne_alt_0_p_0_m2) 
# Defining interpolation function for Ne (as function of time)
interp_fun_Ne = interp1d(time_interp, Ne_interp,kind='cubic')

#Define time array w/ spacing similar to original time sampling
N = Ne_index[0]
n_time_elements = int(N[-1]) - int(N[0])+1
time_array = np.linspace(time_interp[0],time_interp[-1],n_time_elements)

#Define functions used to filter data
def butter_lowpass(cutoff, fs, order):
    nyq = 0.5 * fs                    #Nyquist frequency
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y


# Filter requirements.
order = 6
sampling_period_hr = time_array[1] - time_array[0]
fs = float(1) / float(sampling_period_hr) # sampling frequency, Hz
cutoff = float(1) / float(2) # desired cutoff frequency of the filter, Hz

# Get the filter coefficients so we can check its frequency response.
b, a = butter_lowpass(cutoff, fs, order)

#Setting up (evenly spaced) time and Ne arrays
t = time_array
data = interp_fun_Ne(time_array)

#Filter data
filtered_data = butter_lowpass_filter(data, cutoff, fs, order)